In [ ]:
import os
from glob import glob
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests

def send_to_telegram(*args):

    apiToken = '6134697656:AAFAowB-BC50S3of4BxU2WgGfPLQFLCKdWg'
    chatID = '1213767748'
    apiURL = f'https://api.telegram.org/bot{apiToken}/sendMessage'
    message = " ".join(map(lambda x:str(x),args))
    try:
        response = requests.post(apiURL, json={'chat_id': chatID, 'text': message})
    except Exception as e:
        print(e)

In [ ]:
PROJECT_PATH = "/content/drive/MyDrive/Project - II/Ensemble Files"

In [ ]:
EPOCHS=1
POPULATION_SIZE = 25
NO_OF_ITERATIONS = 20
LOWER_BOUND = 0.0
UPPER_BOUND = 1.0

In [ ]:
# out_vec = np.load("Ensemble/balancedX_256_192.npy")
# out_vec = out_vec.astype("float32")
# out_vec /= 255

In [ ]:
# labels = np.load("Ensemble/balancedY_256_192.npy")

In [ ]:
# print(out_vec.shape)
# print(labels.shape)

In [ ]:
X_train = np.load("/content/drive/MyDrive/Project - II/Balanced Data 256_192/X_test.npy")
X_val = np.load("/content/drive/MyDrive/Project - II/Balanced Data 256_192/X_val.npy")

y_train = np.load("/content/drive/MyDrive/Project - II/Balanced Data 256_192/y_test.npy")
y_val = np.load("/content/drive/MyDrive/Project - II/Balanced Data 256_192/y_val.npy")

In [ ]:
X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

In [ ]:
# X_train = X_train[:1000]
# X_val = X_val[:200]
# y_train = y_train[:1000]
# y_val = y_val[:200]

In [ ]:
from tensorflow.keras.optimizers import *

ranges = {
  "Batch Size": [32,64,128],
  "amsgrad":[True,False],
  "weight_decay":[None,1e-5,1e-6],
  "lr":  np.geomspace(1e-3,1e-5,3),
  "b1":np.arange(0.8,0.96,0.05),
  "b2":np.arange(0.990,0.9991,0.003),
  "epsilon":[1e-7,1e-8],
  "factor":[0.1,0.2,0.5],
  "patience": [2,3,5],
  "cooldown":[2,3,5]
}

SOLUTION_SIZE = len(ranges.keys())

In [ ]:
# Population Initialization
population = np.random.uniform(
  low=LOWER_BOUND,
  high=UPPER_BOUND,
  size=(POPULATION_SIZE, SOLUTION_SIZE)
)
print(population.shape)
print(population[0])

(25, 10)
[0.2149059  0.44113276 0.82625773 0.18097418 0.57050742 0.41529574
 0.81226586 0.53128705 0.99972521 0.01372609]


In [ ]:
# Fitness Function Evaluation
def FitnessFunction(solution):
  solution = np.round(solution, 4)

  index = int(np.round(solution[0] * (len(ranges["Batch Size"]) - 1)))
  batchSizeValue = ranges["Batch Size"][index]

  index = int(np.round(solution[1] * (len(ranges["amsgrad"]) - 1)))
  amsgrad = ranges["amsgrad"][index]

  index = int(np.round(solution[2] * (len(ranges["weight_decay"]) - 1)))
  weight_decay = ranges["weight_decay"][index]

  index = int(np.round(solution[3] * (len(ranges["lr"]) - 1)))
  lr = ranges["lr"][index]

  index = int(np.round(solution[4] * (len(ranges["b1"]) - 1)))
  b1 = ranges["b1"][index]

  index = int(np.round(solution[5] * (len(ranges["b2"]) - 1)))
  b2 = ranges["b2"][index]


  index = int(np.round(solution[6] * (len(ranges["epsilon"]) - 1)))
  epsilon = ranges["epsilon"][index]


  index = int(np.round(solution[7] * (len(ranges["factor"]) - 1)))
  factor = ranges["factor"][index]


  index = int(np.round(solution[8] * (len(ranges["patience"]) - 1)))
  patience = ranges["patience"][index]


  index = int(np.round(solution[9] * (len(ranges["cooldown"]) - 1)))
  cooldown = ranges["cooldown"][index]

  # Training Densenet
  pre_trained_densenet_model = DenseNet201(input_shape=(192, 256, 3), include_top=False, weights="imagenet")

  for layer in pre_trained_densenet_model.layers:
      layer.trainable = False

  last_layer = pre_trained_densenet_model.get_layer('relu')
  last_output = last_layer.output
  x = layers.GlobalMaxPooling2D()(last_output)
  x = layers.Dense(512, activation='relu')(x)
  x = layers.Dropout(0.5)(x)
  x = layers.Dense(7, activation='softmax')(x)

  densenet_model = Model(pre_trained_densenet_model.input, x)
  optimizer = Adam(learning_rate=lr, beta_1=b1, beta_2=b2, epsilon=epsilon, weight_decay=weight_decay, amsgrad=amsgrad)
  densenet_model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])
  
  train_datagen = ImageDataGenerator(rotation_range=60, width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2, fill_mode='nearest')

  train_datagen.fit(X_train)

  val_datagen = ImageDataGenerator()
  val_datagen.fit(X_val)
  epochs = 1
  densenet_model.fit(train_datagen.flow(X_train,y_train, batch_size=batchSizeValue),
                                epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                                verbose = 1, steps_per_epoch=(X_train.shape[0] // batchSizeValue), 
                                validation_steps=(X_val.shape[0] // batchSizeValue))
  
  for layer in pre_trained_densenet_model.layers:
      layer.trainable = True
  optimizer = Adam(learning_rate=lr, beta_1=b1, beta_2=b2, epsilon=epsilon, weight_decay=weight_decay, amsgrad=amsgrad)
  densenet_model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['acc'])
  
  learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=patience, verbose=1, factor=factor, 
                                            min_lr=lr/10, cooldown=cooldown)

  epochs = EPOCHS
  densenet_model.fit(train_datagen.flow(X_train,y_train, batch_size=batchSizeValue),
                                epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                                verbose = 1, steps_per_epoch=(X_train.shape[0] // batchSizeValue),
                                validation_steps=(X_val.shape[0] // batchSizeValue),
                                callbacks=[learning_rate_reduction])
  
  # Training inception
  pre_trained_inception_model = InceptionV3(input_shape=(192, 256, 3), include_top=False, weights="imagenet")

  for layer in pre_trained_inception_model.layers:
      layer.trainable = False

  last_layer = pre_trained_inception_model.get_layer('mixed10')
  last_output = last_layer.output
  x = layers.GlobalMaxPooling2D()(last_output)
  x = layers.Dense(512, activation='relu')(x)
  x = layers.Dropout(0.5)(x)
  x = layers.Dense(7, activation='softmax')(x)

  inception_model = Model(pre_trained_inception_model.input, x)
  optimizer = Adam(learning_rate=lr, beta_1=b1, beta_2=b2, epsilon=epsilon, weight_decay=weight_decay, amsgrad=amsgrad)
  inception_model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])
  
  epochs = 1
  history = inception_model.fit(train_datagen.flow(X_train,y_train, batch_size=batchSizeValue),
                                epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                                verbose = 1, steps_per_epoch=(X_train.shape[0] // batchSizeValue), 
                                validation_steps=(X_val.shape[0] // batchSizeValue))
  
  for layer in pre_trained_inception_model.layers:
      layer.trainable = True

  optimizer = Adam(learning_rate=lr, beta_1=b1, beta_2=b2, epsilon=epsilon, weight_decay=weight_decay, amsgrad=amsgrad)
  inception_model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['acc'])
  
  learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=patience, verbose=1, factor=factor, 
                                            min_lr=lr/10, cooldown=cooldown)

  epochs = EPOCHS
  history = inception_model.fit(train_datagen.flow(X_train,y_train, batch_size=batchSizeValue),
                                epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                                verbose = 1, steps_per_epoch=(X_train.shape[0] // batchSizeValue),
                                validation_steps=(X_val.shape[0] // batchSizeValue),
                                callbacks=[learning_rate_reduction])
  
  # def ensemble(models, model_input):

  #     outputs = [model.outputs[0] for model in models]
  #     y = layers.Average()(outputs)
  #     model = Model(model_input, y, name='ensemble')
  #     return model

  input_shape = X_val[0,:,:,:].shape
  model_input = layers.Input(shape=input_shape)

  dense_output = densenet_model(model_input)
  inception_output = inception_model(model_input)
  y = layers.Average()([dense_output,inception_output])

  # ensemble_model = ensemble([densenet_model, inception_model], model_input)

  ensemble_model = Model(model_input,y,name="ensemble")
  ensemble_model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
  
  keyword = "ensemble-" + "-".join([str(el)[2:] for el in solution])
  checkpointPath = os.path.join(PROJECT_PATH, "Checkpoints", keyword) + ".h5"
  csvLogPath = os.path.join(PROJECT_PATH, "Logs", keyword) + ".csv"

  loss_val, acc_val = ensemble_model.evaluate(X_val, y_val, verbose=2)
  # score = scoreList[0]

  configs = [
    batchSizeValue,
    amsgrad,
    weight_decay,
    lr,
    b1,
    b2,
    epsilon,
    factor,
    patience,
    cooldown,
  ]
  # print(score, configs)
  print("Validation: accuracy = %f ;  loss = %f" % (acc_val, loss_val))
  send_to_telegram("Validation: accuracy = %f ;  loss = %f" % (acc_val, loss_val))
  send_to_telegram("Configs", configs)
  # send_to_telegram(score, configs)

  return acc_val

In [ ]:
# Population Updating
def PopulationUpdating(population, scores, iterationNumber):
  bestIndex = np.argmax(scores)
  bestSolution = population[bestIndex].copy()
  bestScore = scores[bestIndex]

  # MRFO:
  # Write the metaheuristic rules for updating the population.
  newPopulation = population.copy()

  coef = iterationNumber / float(NO_OF_ITERATIONS)
  for i in range(len(population)):
    r = np.random.random(1)
    alpha = 2.0 * r * np.sqrt(np.abs(np.log(r)))
    r1 = np.random.random(1)
    factor = (NO_OF_ITERATIONS - iterationNumber + 1.0) / (NO_OF_ITERATIONS * 1.0)
    beta = 2.0 * np.exp(r1 * factor) * np.sin(2.0 * np.pi * r1)
    if (np.random.random(1) < 0.5):
      if (coef < np.random.random(1)):
        s = np.subtract(UPPER_BOUND, LOWER_BOUND)
        u = np.random.uniform(low=0, high=1, size=SOLUTION_SIZE)
        m = np.multiply(u, s)
        xRand = np.clip(np.add(LOWER_BOUND, m), LOWER_BOUND, UPPER_BOUND)
        if (i == 0):
          newPopulation[i, :] = xRand + r * (xRand - population[i, :]) + beta * (xRand - population[i, :])
        else:
          newPopulation[i, :] = xRand + r * (population[i - 1, :] - population[i, :]) + beta * (xRand - population[i, :])
      else:
        if (i == 0):
          newPopulation[i, :] = bestSolution + r * (bestSolution - population[i, :]) + beta * (bestSolution - population[i, :])
        else:
          newPopulation[i, :] = bestSolution + r * (population[i - 1, :] - population[i, :]) + beta * (bestSolution - population[i, :])
    else:
      if (i == 0):
        newPopulation[i, :] = population[i, :] + r * (bestSolution - population[i, :]) + alpha * (bestSolution - population[i, :])
      else:
        newPopulation[i, :] = population[i, :] + r * (population[i - 1, :] - population[i, :]) + alpha * (bestSolution - population[i, :])
    
    newPopulation[i, :] = np.clip(newPopulation[i, :], LOWER_BOUND, UPPER_BOUND)
    
    currentScore = FitnessFunction(newPopulation[i, :])
    if (currentScore > bestScore):
      bestSolution, bestScore = newPopulation[i, :].copy(), currentScore
    
    s = 2.0
    r2, r3 = np.random.random(1), np.random.random(1)
    newPopulation[i, :] = population[i, :] + s * (r2 * bestSolution - r3 * population[i, :])
    
    newPopulation[i, :] = np.clip(newPopulation[i, :], LOWER_BOUND, UPPER_BOUND)
    
    currentScore = FitnessFunction(newPopulation[i, :])
    if (currentScore > bestScore):
      bestSolution, bestScore = newPopulation[i, :].copy(), currentScore

  return newPopulation.copy()

In [ ]:
# Repeat
bestSolutions = []
bestScores = []
for iterationNumber in range(NO_OF_ITERATIONS):
  scores = []
  send_to_telegram(f"{iterationNumber} Started")
  for i in range(len(population)):
    print(f"iteration-{iterationNumber} population-{i}")
    score = FitnessFunction(population[i])
    send_to_telegram(f"iteration-{iterationNumber} population-{i} Done")
    scores.append(score)

  newPopulation = PopulationUpdating(population, scores, iterationNumber)
  
  # LOGGING THE DATA
  populationScoresPath = os.path.join(PROJECT_PATH, "Population.csv")
  # T #, S #, ......, Score
  file = open(populationScoresPath, "a")
  for i in range(len(population)):
    data = f"{iterationNumber + 1},{i + 1},"
    data += ",".join([str(el) for el in population[i]])
    data += f",{scores[i]}"
    data += "\n"
    file.write(data)
  file.close()

  bestIndex = np.argmax(scores)
  bestSolution = population[bestIndex].copy()
  bestScore = scores[bestIndex]
  bestSolutions.append(bestSolution)
  bestScores.append(bestScore)
  
  population = newPopulation.copy()
     

iteration-0 population-0
31/31 [==============================] - 54s 485ms/step - loss: 1.8423 - acc: 0.2603 - val_loss: 13.2678 - val_acc: 0.1198 - lr: 0.0010
7/7 - 10s - loss: 8.9434 - accuracy: 0.1150 - 10s/epoch - 1s/step
Validation: accuracy = 0.115000 ;  loss = 8.943394
iteration-0 population-1
31/31 [==============================] - 55s 486ms/step - loss: 2.0131 - acc: 0.1983 - val_loss: 2598.9534 - val_acc: 0.1250 - lr: 0.0010
7/7 - 9s - loss: 12.4161 - accuracy: 0.1250 - 9s/epoch - 1s/step
Validation: accuracy = 0.125000 ;  loss = 12.416108
iteration-0 population-2
31/31 [==============================] - 68s 482ms/step - loss: 3.6971 - acc: 0.1519 - val_loss: 2.6250 - val_acc: 0.1302 - lr: 1.0000e-05


In [ ]:
# LOGGING THE DATA
bestSolutionsPath = os.path.join(PROJECT_PATH, "BestSolutions.csv")
file = open(bestSolutionsPath, "w")
for i in range(len(bestSolutions)):
  data = ",".join([str(el) for el in bestSolutions[i]])
  data += f",{bestScores[i]}"
  data += "\n"
  file.write(data)
file.close()

In [ ]:
print(bestSolutions)

In [ ]:
import pandas as pd

In [ ]:
bestSolutions = pd.read_csv(os.path.join(PROJECT_PATH,"BestSolutions.csv"), header=None)

In [ ]:
bestSolutions

In [ ]:
bestSolutions = bestSolutions.to_numpy()

In [ ]:
bestSolutions.shape

In [ ]:
import numpy as np

In [ ]:
for i in range(len(bestSolutions)):
    solution = np.round(bestSolutions[i], 4)

    index = int(np.round(solution[0] * (len(ranges["Batch Size"]) - 1)))
    batchSizeValue = ranges["Batch Size"][index]

    index = int(np.round(solution[1] * (len(ranges["amsgrad"]) - 1)))
    amsgrad = ranges["amsgrad"][index]

    index = int(np.round(solution[2] * (len(ranges["weight_decay"]) - 1)))
    weight_decay = ranges["weight_decay"][index]

    index = int(np.round(solution[3] * (len(ranges["lr"]) - 1)))
    lr = ranges["lr"][index]

    index = int(np.round(solution[4] * (len(ranges["b1"]) - 1)))
    b1 = ranges["b1"][index]

    index = int(np.round(solution[5] * (len(ranges["b2"]) - 1)))
    b2 = ranges["b2"][index]


    index = int(np.round(solution[6] * (len(ranges["epsilon"]) - 1)))
    epsilon = ranges["epsilon"][index]


    index = int(np.round(solution[7] * (len(ranges["factor"]) - 1)))
    factor = ranges["factor"][index]


    index = int(np.round(solution[8] * (len(ranges["patience"]) - 1)))
    patience = ranges["patience"][index]


    index = int(np.round(solution[9] * (len(ranges["cooldown"]) - 1)))
    cooldown = ranges["cooldown"][index]
    
    configs = [
      batchSizeValue,
      amsgrad,
      weight_decay,
      lr,
      b1,
      b2,
      epsilon,
      factor,
      patience,
      cooldown,
    ]
    print(configs, "Accuarcy =", solution[10])